# RLHF-PPO Pytorch Implemention

Author: *xiaodongguaAIGC*

Github: *dhcode-cpp*

# define basic function

## config

In [1]:
vocab_size = 32
hidden_size = 256
intermediate_size = 512
num_hidden_layers = 2
num_attention_heads = 4
num_key_value_heads = 4
batch_size = 2
length_x = 6
max_new_tokens = 5

## pretrained model

In [2]:
import torch
import torch.nn.functional as F
from transformers import LlamaModel, LlamaConfig, LlamaForCausalLM, LlamaForSequenceClassification

torch.manual_seed(1)

# 加载模型
config = LlamaConfig(vocab_size=vocab_size,      # default is 32000
                     hidden_size=hidden_size,
                     intermediate_size=intermediate_size,
                     num_hidden_layers=num_hidden_layers,
                     num_attention_heads=num_attention_heads,
                     num_key_value_heads=num_key_value_heads,
                     )
model = LlamaForCausalLM(config)
model.config.pad_token_id = model.config.eos_token_id
model.save_pretrained('./lm_pretrained')

In [5]:
x = torch.randint(0, vocab_size, (batch_size, length_x))
xy = torch.randint(0, vocab_size, (batch_size, length_x + max_new_tokens))
print(x.shape)
print(x)

torch.Size([2, 6])

tensor([[ 7, 17,  6,  1, 11, 28],
        [11,  5, 30, 22, 18,  5]])

In [6]:
mask_zero = torch.zeros(batch_size, length_x)
mask = torch.ones(batch_size, length_x+max_new_tokens)
mask[:, :length_x] = mask_zero
print(mask)

tensor([[0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]])

## lm model generate

https://huggingface.co/docs/transformers/main_classes/text_generation

In [7]:
def get_generate(model, x, max_new_tokens):
    idx = {'input_ids': x}  # ignore mask
    y = model.generate(**idx,
                       max_new_tokens=max_new_tokens,
                       forced_eos_token_id=True)
    return y


print('input:', x)
gen_xy = get_generate(model, x, max_new_tokens)
print('lm output:\n', gen_xy)
print('lm output shape:\n', gen_xy.shape)

input: tensor([[ 7, 17,  6,  1, 11, 28],
        [11,  5, 30, 22, 18,  5]])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


lm output:
 tensor([[ 7, 17,  6,  1, 11, 28, 30, 20,  6, 30,  0],
        [11,  5, 30, 22, 18,  5, 26, 26, 26, 26,  0]])

lm output shape:

torch.Size([2, 11])

## reward model

In [8]:
model_rm = LlamaForSequenceClassification.from_pretrained(
    './lm_pretrained', num_labels=1)
print(model_rm)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at ./lm_pretrained and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32, 256, padding_idx=2)
    (layers): ModuleList(
      (0-1): 2 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=256, out_features=256, bias=False)
          (k_proj): Linear(in_features=256, out_features=256, bias=False)
          (v_proj): Linear(in_features=256, out_features=256, bias=False)
          (o_proj): Linear(in_features=256, out_features=256, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=256, out_features=512, bias=False)
          (up_proj): Linear(in_features=256, out_features=512, bias=False)
          (down_proj): Linear(in_features=512, out_features=256, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (score): Linear(in_features=256, out_features=1, bias=False)
)

## rm get reward

In [11]:
def get_reward(rm_model, x):
    idx = {'input_ids': x}  # ignore mask
    y = model_rm(**idx).logits
    return y


print(gen_xy)
rm_xy = get_reward(model_rm, gen_xy)
print(rm_xy)

tensor([[ 7, 17,  6,  1, 11, 28, 30, 20,  6, 30,  0],
        [11,  5, 30, 22, 18,  5, 26, 26, 26, 26,  0]])

tensor([[0.4214],
        [0.7454]], grad_fn=<IndexBackward0>)

# PPO Pipeline

### ppo config

In [12]:
class PPOConfig():
    def __init__(self):
        self.ppo_epochs = 3
        self.mini_batch_size = 1
        self.epochs = 2
        self.kl_ctl = 0.01
        self.vl_coef = 0.01
        self.lam = 0.9
        self.gamma = 0.9
        self.cliprange_value = 0.2
        self.cliprange = 0.2
        # self.param = {'gamma':0.9, 'lambda':0.9,}

    def __str__(self):
        return f'ppo_epochs:{self.ppo_epochs}\nmini_batch_size:{self.mini_batch_size}\nepochs:{self.epochs}\nkl_ctl:{self.kl_ctl}'


ppo_config = PPOConfig()
print(ppo_config)

ppo_epochs:3
mini_batch_size:1
epochs:2
kl_ctl:0.01

### PPO models

In [13]:
# model -> [sft] -> model_ref
# model_ref -> model_rm
# model_ref -> model_actor
# model_rm -> model_critic
model_ref = LlamaForCausalLM(config)
# model_critic = LlamaForSequenceClassification.from_pretrained('./lm_pretrained', num_labels=1)

### model critic

In [14]:
config = LlamaConfig(vocab_size=vocab_size,      # default is 32000
                     hidden_size=hidden_size,
                     intermediate_size=intermediate_size,
                     num_hidden_layers=num_hidden_layers,
                     num_attention_heads=num_attention_heads,
                     num_key_value_heads=num_key_value_heads,
                     )
model_base = LlamaForCausalLM(config)
model_base.config.pad_token_id = model_base.config.eos_token_id


class ModelValueHead(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.dropout = torch.nn.Dropout(0.05)
        self.summary = torch.nn.Linear(model.config.hidden_size, 1)

    def forward(self, xy):
        hidden_states = model(**xy, output_hidden_states=True).hidden_states
        last_hidden_states = hidden_states[-1]
        # print(last_hidden_states.shape)
        output = self.dropout(last_hidden_states)
        output = self.summary(output)[:, :, 0]
        return output


def get_values(model, xy):
    idx = {'input_ids': xy}
    value = model(idx)
    return value


model_critic = ModelValueHead(model_base)

print(xy)
value = get_values(model_critic, xy)
print(value)

tensor([[ 3,  2, 21, 16, 31, 21, 18, 27,  8, 11,  4],
        [ 7,  3, 11, 27, 13,  4, 19, 29, 26,  3, 23]])

tensor([[ 0.6379,  0.8124,  0.9475,  1.3293,  1.5653,  1.8487,  1.4692,  1.2117,
          1.6160,  1.2856,  0.9910],
        [-0.6929, -0.1668,  0.3139, -0.2659,  0.3015,  0.2497, -0.0748, -0.0441,
         -0.1106,  0.0822,  0.0668]], grad_fn=<SelectBackward0>)

In [15]:
class PPOModels():
    def __init__(self, model_actor, model_ref, model_rm, model_critic):
        self.actor = model_actor
        self.ref = model_ref
        self.rm = model_rm
        self.critic = model_critic


model_ref.eval()
model_rm.eval()
models = PPOModels(model, model_ref, model_rm, model_critic)

## PPO Pipeline

In [16]:
def void_ppo_step(models, ppo_config, x, xy, rewards):
    # just for show pipeline
    return 0.1, 0.2

In [17]:
def ppo_train(models,
              ppo_config,
              ppo_step,
              x):
    for i in range(ppo_config.epochs):
        # use dataloader get batch
        print(x)
        xy = get_generate(models.actor, x, max_new_tokens)
        print(xy)
        rewards = get_reward(models.rm, xy)
        print(rewards)
        loss, reward = ppo_step(models, ppo_config, x, xy, rewards)
    return loss, reward


loss, reward = ppo_train(models, ppo_config, void_ppo_step, x)
print(loss)
print(reward)

tensor([[ 7, 17,  6,  1, 11, 28],
        [11,  5, 30, 22, 18,  5]])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[ 7, 17,  6,  1, 11, 28, 30, 20,  6, 30,  0],
        [11,  5, 30, 22, 18,  5, 26, 26, 26, 26,  0]])

tensor([[0.4214],
        [0.7454]], grad_fn=<IndexBackward0>)

tensor([[ 7, 17,  6,  1, 11, 28],
        [11,  5, 30, 22, 18,  5]])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[ 7, 17,  6,  1, 11, 28, 30, 20,  6, 30,  0],
        [11,  5, 30, 22, 18,  5, 26, 26, 26, 26,  0]])

tensor([[0.4214],
        [0.7454]], grad_fn=<IndexBackward0>)

0.1

0.2

## ppo_step pipeline

1. make experience
2. ppo_train

### define ppo experience dict

In [18]:
ppo_batchs = {
    'prompt': None,
    'sequence': None,
    'mask': None,
    'logprobs_ref': None,
    'logprobs_old': None,
    'logprobs': None,
    'values_old': None,
    'values': None,
    'rewards': None,
    'rewards_kl': None,
    'loss': None,
    'logits': None,
}

ppo_batchs['prompt'] = x
ppo_batchs['sequence'] = xy
ppo_batchs['mask'] = mask

### Forward Get Policy

In [19]:
def logprobs_from_logits(logits, labels, gather=True):
    logp = F.log_softmax(logits, dim=2)
    logpy = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logpy, logp


# get policy
logits = torch.randn(1, 5, vocab_size)
labels = torch.tensor([[3, 2, 4, 3, 2]])
logpy, logp = logprobs_from_logits(logits, labels)

# get prob
idx = 2
print('one of logits:', logits[:, idx, :])
print('one of lobp:', logp[:, idx, :])
print('logp get labels[-1]', logp[:, idx, labels[:, idx]])
print('one of logpy:', logpy[:, idx])

one of logits: tensor([[ 0.7250, -0.4507,  1.0910, -0.3271, -1.2161, -0.8386, -2.1642, -0.4984,
          1.7428,  0.7939, -0.3338,  0.1734, -0.9755,  1.3061,  0.7058, -0.8236,
         -0.0927,  0.2694,  1.0248, -0.3739,  0.5990,  0.2219, -1.8697,  1.4419,
          0.1668,  0.1829,  0.2971,  1.3877,  2.6458, -0.5310, -1.2883, -0.4366]])

one of lobp: tensor([[-3.3810, -4.5566, -3.0149, -4.4330, -5.3220, -4.9445, -6.2701, -4.6044,
         -2.3632, -3.3121, -4.4397, -3.9325, -5.0814, -2.7999, -3.4001, -4.9296,
         -4.1986, -3.8365, -3.0811, -4.4798, -3.5069, -3.8841, -5.9757, -2.6641,
         -3.9392, -3.9230, -3.8089, -2.7182, -1.4601, -4.6370, -5.3942, -4.5426]])

logp get labels[-1] tensor([[-5.3220]])

one of logpy: tensor([-5.3220])

In [20]:
def get_logits(model, xy):
    idx = {'input_ids': xy}  # ignore mask
    logits = model(**idx).logits
    return logits


print(xy)
logits = get_logits(models.actor, xy)
print(logits.shape)
probs, _ = logprobs_from_logits(logits, xy, True)
print(probs.shape)

tensor([[ 3,  2, 21, 16, 31, 21, 18, 27,  8, 11,  4],
        [ 7,  3, 11, 27, 13,  4, 19, 29, 26,  3, 23]])

torch.Size([2, 11, 32])

torch.Size([2, 11])

### PPO Forward Code

In [21]:
def batch_forward(model_policy, model_value, xy):
    logits = get_logits(model_policy, xy)
    logprobs, _ = logprobs_from_logits(logits, xy, True)
    values = None
    if model_value != None:
        values = get_values(model_value, xy)
    return logprobs, values, logits


with torch.no_grad():
    logprobs_ref, _, _ = batch_forward(models.ref, None, xy)
logprobs_old, values_old, _ = batch_forward(models.actor, models.critic, xy)

print(logprobs_ref.shape)
print(logprobs_old.shape)
print(values_old.shape)

ppo_batchs['logprobs_ref'] = logprobs_ref
ppo_batchs['logprobs_old'] = logprobs_old
ppo_batchs['values_old'] = values_old

torch.Size([2, 11])

torch.Size([2, 11])

torch.Size([2, 11])

### reward with KL

In [33]:
def compute_rewards_kl(
    reward,
    ref_logprobs,
    old_logprobs,
    kl_ctl,
):
    kl = logprobs_old[:, :] - logprobs_ref[:, :]
    kl_reward = -kl_ctl * kl
    kl_reward[:, -1] += reward[:, 0]
    return kl_reward


kl_ctl = 0.01
score = 100.0  # obeserver the value
kl = logprobs_old - logprobs_ref
reward = -kl_ctl * kl
print('only kl')
print(reward[0, :])
print(reward[0, -1])
print('reward with kl')
reward[:, -1] += (rm_xy[:, 0] + score)
print(reward[0, :])
print(reward[0, -1])

print('compute rewards with kl for PPO')
rm_xy = torch.randn(2, 1)
reward_kl = compute_rewards_kl(rm_xy, logprobs_ref, logprobs_old, 0.01)
print(reward_kl)
ppo_batchs['rewards'] = rm_xy
ppo_batchs['rewards_kl'] = reward_kl

only kl

tensor([-0.0011, -0.0009,  0.0033, -0.0012,  0.0006,  0.0029,  0.0013, -0.0078,
        -0.0012, -0.0053, -0.0019], grad_fn=<SliceBackward0>)

tensor(-0.0019, grad_fn=<SelectBackward0>)

reward with kl

tensor([-1.0923e-03, -8.7936e-04,  3.2610e-03, -1.2097e-03,  6.1776e-04,
         2.8765e-03,  1.2557e-03, -7.8435e-03, -1.1751e-03, -5.2777e-03,
         1.0046e+02], grad_fn=<SliceBackward0>)

tensor(100.4612, grad_fn=<SelectBackward0>)

compute rewards with kl for PPO

tensor([[-0.0011, -0.0009,  0.0033, -0.0012,  0.0006,  0.0029,  0.0013, -0.0078,
         -0.0012, -0.0053, -0.0030],
        [ 0.0038,  0.0049, -0.0026, -0.0017, -0.0022,  0.0016, -0.0026,  0.0027,
         -0.0014,  0.0050,  0.0040]], grad_fn=<CopySlices>)

about reward tips

pad token id = 1

input :

x1 : 3, 4, 6, 8, 20, 29, 30, 2

x2 : 8, 2, 6, 9, 13, 2,  1, 1

x1 mask:

x1 : 1, 1, 1, 1, 1,  1,  1,  1

x2 mask:

x1 : 1, 1, 1, 1, 1,  1,  0,  0

KL+rewards

x1 : 1, 1, 1, 1, 1,  1,  1,  [1]+rewards

x1 : 1, 1, 1, 1, 1,  [1]+rewards,  0,  0

### data before ppo_train

In [34]:
'''
1. we use forward get policy,policy_old,value,
2. we compute reward with kl
'''


def ppo_prepare(models, ppo_config, xy, rm_xy):
    with torch.no_grad():
        logprobs_ref, _ = batch_forward(models.ref, None, xy)
    logprobs_old, values_old = batch_forward(models.actor, models.critic, xy)
    reward_kl = compute_rewards_kl(
        rm_xy, logprobs_ref, logprobs_old, ppo_config.kl_ctl)
    return logprobs_old, values_old, logprobs_ref, reward_kl

# logprobs_old, values_old, logprobs_ref, reward_kl = ppo_prepare(models, ppo_config, xy, rm_xy)


ppo_batchs


{
│   'prompt': tensor([[ 7, 17,  6,  1, 11, 28],
│   │   [11,  5, 30, 22, 18,  5]]),
│   'sequence': tensor([[ 3,  2, 21, 16, 31, 21, 18, 27,  8, 11,  4],
│   │   [ 7,  3, 11, 27, 13,  4, 19, 29, 26,  3, 23]]),
│   'mask': tensor([[0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
│   │   [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]]),
│   'logprobs_ref': tensor([[-3.2797, -3.9794, -3.3771, -3.5436, -3.6296, -3.3818, -3.8935, -3.7257,
│   │    -3.7764, -3.2362, -3.6417],
│   │   [-2.9919, -3.1163, -3.1112, -3.7207, -3.5101, -3.5335, -3.5303, -3.1816,
│   │    -3.3280, -3.0611, -3.6552]]),
│   'logprobs_old': tensor([[-3.1704, -3.8914, -3.7032, -3.4227, -3.6914, -3.6695, -4.0191, -2.9413,
│   │    -3.6589, -2.7084, -3.4496],
│   │   [-3.3709, -3.6106, -2.8478, -3.5536, -3.2888, -3.6936, -3.2728, -3.4517,
│   │    -3.1879, -3.5649, -3.6766]], grad_fn=<SqueezeBackward1>),
│   'logprobs': tensor([[-3.1704, -3.8914, -3.7032, -3.4227, -3.6914, -3.6695, -4.0191, -2.9413,
│   │    -3.6589, -2.708

# PPO Train Step

In [35]:
# if batch size is 32
# the mini_bacth_size = 8
# like data loader would run 4 step
def get_minibatch(ppo_batchs, batch_size, mini_batch_size):
    step = batch_size // mini_batch_size
    ppo_batchs_iter = []
    print(step)
    for i in range(step):
        ppo_batchs_iter.append(ppo_batchs)
    return ppo_batchs_iter


def void_compute_loss(models, mini_batchs, ppo_config):
    print('roll mini batch to compute ppo loss')
    print(mini_batchs)
    loss = 1.0
    pl_loss = 2.0
    vl_loss = 3.0
    loss = pl_loss + ppo_config.vl_coef * vl_loss
    return loss, pl_loss, vl_loss


def ppo_train_step(models, ppo_batchs, ppo_config, compute_loss):
    losses = []
    for i in range(ppo_config.ppo_epochs):
        ppo_batchs_iter = get_minibatch(
            ppo_batchs, batch_size, ppo_config.mini_batch_size)
        for mini_batchs in ppo_batchs_iter:
            # get training policy
            logprobs, values, logits = batch_forward(
                models.actor, models.critic, mini_batchs['sequence'])
            mini_batchs['logprobs'] = logprobs
            mini_batchs['values'] = values
            mini_batchs['logits'] = logits
            # compute loss
            loss, pl_loss, vl_loss = compute_loss(
                models, mini_batchs, ppo_config)
            # loss.backward()
            losses.append(loss)
            break  # just for debug,
        break  # just for debug,
        mini_batchs['loss'] = losses
    return losses


loss = ppo_train_step(models, ppo_batchs, ppo_config, void_compute_loss)
print(torch.tensor(loss).mean())

2

roll mini batch to compute ppo loss

{
    'prompt': tensor([[ 7, 17,  6,  1, 11, 28],
        [11,  5, 30, 22, 18,  5]]),
    'sequence': tensor([[ 3,  2, 21, 16, 31, 21, 18, 27,  8, 11,  4],
        [ 7,  3, 11, 27, 13,  4, 19, 29, 26,  3, 23]]),
    'mask': tensor([[0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]]),
    'logprobs_ref': tensor([[-3.2797, -3.9794, -3.3771, -3.5436, -3.6296, -3.3818, -3.8935, -3.7257,
         -3.7764, -3.2362, -3.6417],
        [-2.9919, -3.1163, -3.1112, -3.7207, -3.5101, -3.5335, -3.5303, -3.1816,
         -3.3280, -3.0611, -3.6552]]),
    'logprobs_old': tensor([[-3.1704, -3.8914, -3.7032, -3.4227, -3.6914, -3.6695, -4.0191, -2.9413,
         -3.6589, -2.7084, -3.4496],
        [-3.3709, -3.6106, -2.8478, -3.5536, -3.2888, -3.6936, -3.2728, -3.4517,
         -3.1879, -3.5649, -3.6766]], grad_fn=<SqueezeBackward1>),
    'logprobs': tensor([[-3.1704, -3.8914, -3.7032, -3.4227, -3.6914, -3.6695, -4.0191, -2.9413,
         -3.6589, -2.7084, -3.4496],
        [-3.3709, -3.6106, -2.8478, -3.5536, -3.2888, -3.6936, -3.2728, -3.4517,
         -3.1879, -3.5649, -3.6766]], grad_fn=<SqueezeBackward1>),
    'values_old': tensor([[ 0.4037,  0.5650,  1.1206,  1.3727,  1.5973,  1.4748,  1.5315,  1.2411,
          1.4551,  1.3217,  1.1747],
        [-0.5438, -0.1715,  0.0838, -0.4835,  0.3192,  0.2480, -0.1122,  0.0394,
         -0.4591,  0.0104,  0.0802]], grad_fn=<SelectBackward0>),
    'values': tensor([[ 0.5405,  0.8507,  1.3907,  1.0501,  1.4372,  1.6204,  1.7181,  1.1506,
          1.5603,  1.2491,  1.0641],
        [-0.5455, -0.1241, -0.0128, -0.2249, -0.0358, -0.0632, -0.1209,  0.0357,
         -0.2630,  0.2597,  0.2625]], grad_fn=<SelectBackward0>),
    'rewards': tensor([[-0.4233],
        [-0.3428]]),
    'rewards_kl': tensor([[-0.0011, -0.0009,  0.0033, -0.0012,  0.0006,  0.0029,  0.0013, -0.0078,
         -0.0012, -0.0053, -0.0030],
        [ 0.0038,  0.0049, -0.0026, -0.0017, -0.0022,  0.0016, -0.0026,  0.0027,
         -0.0014,  0.0050,  0.0040]], grad_fn=<CopySlices>),
    'loss': None,
    'logits': tensor([[[-2.9141e-01, -2.3078e-01, -4.4092e-01,  3.1467e-01,  7.3815e-02,
           2.3336e-01, -1.0261e-01, -5.1662e-01,  1.0788e-01, -5.3807e-02,
           6.6215e-02,  1.9772e-01, -1.1797e-01, -6.8881e-02,  3.0796e-02,
           1.5639e-01,  1.4710e-01, -6.5384e-02, -3.0084e-01,  4.0951e-01,
           6.0962e-01,  2.0662e-01, -3.6570e-01,  2.2405e-02, -4.3202e-02,
          -3.0117e-01,  2.6883e-02,  1.5225e-01, -4.2351e-01,  1.1622e-01,
           2.0323e-01, -1.7224e-01],
         [ 1.7116e-01, -1.3915e-01, -3.7983e-01,  2.3101e-01, -2.0881e-01,
           1.6951e-01,  1.7153e-01, -8.2568e-01,  1.2967e-01, -2.8406e-01,
          -2.7890e-02,  6.8381e-01,  1.9228e-01, -3.4845e-02, -2.2157e-01,
           2.4607e-01,  4.0112e-02,  3.2355e-01, -4.3763e-01, -2.5645e-02,
           3.9134e-01, -2.4255e-01,  1.4322e-01,  7.6041e-04,  2.2418e-01,
          -3.2726e-01,  2.1264e-01,  3.6522e-01, -3.8521e-01,  1.3221e-01,
           3.7866e-03, -1.7415e-01],
         [ 2.2526e-01, -2.0581e-01, -3.5515e-01,  4.1750e-02,  2.0791e-02,
           2.6506e-01,  3.1328e-01, -4.9314e-01,  2.3665e-01,  6.5306e-02,
          -4.4244e-01,  7.2106e-01,  2.3617e-01, -1.8132e-01, -1.9663e-01,
           2.3336e-01,  2.6515e-02,  3.3128e-01, -6.1750e-01, -1.3576e-01,
           3.9121e-01, -1.7041e-01,  1.3463e-01, -2.7857e-01,  3.6622e-01,
          -1.9625e-01,  6.8053e-02,  7.2521e-01, -7.3560e-01,  2.8994e-01,
          -1.0810e-01, -3.2995e-01],
         [ 2.5728e-01, -2.3800e-01, -3.6303e-01, -7.6463e-02,  6.4079e-03,
           9.7700e-02, -8.2348e-02, -5.9503e-01,  3.5240e-02,  2.4716e-02,
          -2.6025e-01,  7.7423e-01,  2.9201e-01, -1.3588e-01, -3.4053e-02,
           3.8255e-01,  7.2180e-02,  2.8551e-01, -8.3311e-01, -1.6609e-01,
           1.6344e-01, -1.6694e-01,  8.9974e-02, -5.5195e-01,  1.5331e-01,
          -1.6975e-01,  3.3836e-01,  7.3927e-01, -6.7911e-01, -2.2448e-02,
  

tensor(2.0300)

### Compute ppo loss

when calulative ppo loss in TOKEN-LEVEL
PPO loss pipeline:
1. get GAE
2. get Critic Value loss
3. get Policy loss
4. get entropy loss

#### Get GAE

In [36]:
def get_GAE(rewards, mask, values, gamma, lam):
    lastgaelam = 0
    advantages_reversed = []
    gen_len = rewards.shape[-1]
    print(gen_len)

    values = values * mask
    rewards = rewards * mask

    for t in reversed(range(gen_len)):
        nextvalues = values[:, t + 1] if t < gen_len - 1 else 0.0
        delta = rewards[:, t] + gamma * nextvalues - values[:, t]
        lastgaelam = delta + gamma * lam * lastgaelam
        advantages_reversed.append(lastgaelam)
    advantages = torch.stack(advantages_reversed[::-1]).transpose(0, 1)
    return advantages


# for oberserve gae data
mini_batchs = ppo_batchs.copy()
tmp_reward = torch.ones_like(mini_batchs['rewards_kl']) * 10.0
tmp_reward[:, :-1] = 0.0
print(tmp_reward)
tmp_value = torch.ones_like(mini_batchs['values_old']) * 0.0

result = get_GAE(tmp_reward,
                 mini_batchs['mask'],
                 tmp_value,
                 0.9,
                 0.9)
print(result)


# for oberserve gae data
mini_batchs = ppo_batchs.copy()
print(mini_batchs['rewards_kl'])
tmp_reward = torch.ones_like(mini_batchs['rewards_kl']) * 10.0
tmp_reward[:, :-1] = 0.0
print(tmp_reward)
tmp_value = torch.ones_like(mini_batchs['values_old']) * 1.0

result = get_GAE(tmp_reward,
                 mini_batchs['mask'],
                 tmp_value,
                 0.9,
                 0.9)
print(result)

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.]])

11

tensor([[ 1.2158,  1.5009,  1.8530,  2.2877,  2.8243,  3.4868,  4.3047,  5.3144,
          6.5610,  8.1000, 10.0000],
        [ 1.2158,  1.5009,  1.8530,  2.2877,  2.8243,  3.4868,  4.3047,  5.3144,
          6.5610,  8.1000, 10.0000]])

tensor([[-0.0011, -0.0009,  0.0033, -0.0012,  0.0006,  0.0029,  0.0013, -0.0078,
         -0.0012, -0.0053, -0.0030],
        [ 0.0038,  0.0049, -0.0026, -0.0017, -0.0022,  0.0016, -0.0026,  0.0027,
         -0.0014,  0.0050,  0.0040]], grad_fn=<CopySlices>)

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.]])

11

tensor([[1.3233, 1.6338, 2.0170, 2.4901, 3.0742, 3.7953, 3.5745, 4.5364, 5.7239,
         7.1900, 9.0000],
        [1.3233, 1.6338, 2.0170, 2.4901, 3.0742, 3.7953, 3.5745, 4.5364, 5.7239,
         7.1900, 9.0000]])

In [37]:
# use mini_batchs data
mini_batchs = ppo_batchs.copy()
GAE = get_GAE(mini_batchs['rewards_kl'],
              mini_batchs['mask'],
              mini_batchs['values'],
              ppo_config.gamma,
              ppo_config.lam)
print(GAE)

11

tensor([[ 1.4768e-01,  1.8232e-01,  2.2508e-01,  2.7788e-01,  3.4306e-01,
          4.2353e-01, -1.3861e+00, -8.7015e-01, -1.3777e+00, -1.1611e+00,
         -1.0671e+00],
        [ 4.4890e-04,  5.5420e-04,  6.8420e-04,  8.4469e-04,  1.0428e-03,
          1.2874e-03,  1.3593e-01, -1.7887e-02,  3.1078e-01, -2.2783e-01,
         -2.5848e-01]], grad_fn=<TransposeBackward0>)

#### Get Critic Value loss

In [38]:
def masked_mean(values, mask, axis=None):
    if axis is not None:
        return (values * mask).sum(axis=axis) / mask.sum(axis=axis)
    else:
        return (values * mask).sum() / mask.sum()


def clip_by_value(x, tensor_min, tensor_max):
    clipped = torch.max(torch.min(x, tensor_max), tensor_min)
    return clipped


def get_value_loss(advantages, values, values_old, mask, cliprange_value):
    returns = advantages + values_old
    advantages = advantages.detach()
    vpredclipped = clip_by_value(
        values,
        values_old - cliprange_value,
        values_old + cliprange_value,
    )

    vf_losses1 = (values - returns) ** 2
    vf_losses2 = (vpredclipped - returns) ** 2

    # print("vf_losses2:", vf_losses2)
    vf_loss = 0.5 * masked_mean(torch.max(vf_losses1, vf_losses2), mask)
    vf_clipfrac = masked_mean(torch.gt(vf_losses2, vf_losses1).float(), mask)
    return vf_loss


vf_loss = get_value_loss(GAE,
                         mini_batchs['values'],
                         mini_batchs['values_old'],
                         mini_batchs['mask'],
                         ppo_config.cliprange_value)
print(vf_loss)

tensor(0.3918, grad_fn=<MulBackward0>)

#### Get Policy loss

In [39]:
def get_policy_loss(logprobs, lobprobs_old, advantages, mask, cliprange):
    ratio = torch.exp(logprobs - lobprobs_old)

    pg_losses = -advantages * ratio
    pg_losses2 = -advantages * \
        torch.clamp(ratio, 1.0 - cliprange, 1.0 + cliprange)

    pg_loss = masked_mean(torch.max(pg_losses, pg_losses2), mask)
    pg_clipfrac = masked_mean(torch.gt(pg_losses2, pg_losses).float(), mask)
    return pg_loss


pg_loss = get_policy_loss(mini_batchs['logprobs'],
                          mini_batchs['logprobs_old'],
                          GAE,
                          mini_batchs['mask'],
                          ppo_config.cliprange,)
print(pg_loss)

tensor(0.5920, grad_fn=<DivBackward0>)

#### Get Entropy loss

In [40]:
def get_entropy_loss(logits, mask):
    pd = torch.nn.functional.softmax(logits, dim=-1)
    entropy = torch.logsumexp(logits, axis=-1) - \
        torch.sum(pd * logits, axis=-1)
    return entropy


entropy = get_entropy_loss(mini_batchs['logits'], mini_batchs['mask'])
print(entropy)

tensor([[3.4332, 3.4250, 3.4069, 3.4029, 3.4065, 3.3979, 3.4009, 3.4084, 3.4109,
         3.4030, 3.4119],
        [3.4266, 3.4265, 3.4223, 3.4235, 3.4291, 3.4290, 3.4217, 3.4281, 3.4214,
         3.4151, 3.4233]], grad_fn=<SubBackward0>)

### gather all loss

In [41]:
def compute_loss(mini_batchs, ppo_config):

    GAE = get_GAE(mini_batchs['rewards_kl'],
                  mini_batchs['mask'],
                  mini_batchs['values'],
                  ppo_config.gamma,
                  ppo_config.lam)

    vl_loss = get_value_loss(GAE,
                             mini_batchs['values'],
                             mini_batchs['values_old'],
                             mini_batchs['mask'],
                             ppo_config.cliprange_value)

    pg_loss = get_policy_loss(mini_batchs['logprobs'],
                              mini_batchs['logprobs_old'],
                              GAE,
                              mini_batchs['mask'],
                              ppo_config.cliprange,)

    loss = pg_loss + ppo_config.vl_coef * vl_loss

    return loss, pg_loss, vl_loss


mini_batchs = ppo_batchs.copy()
loss, _, _ = compute_loss(mini_batchs, ppo_config)
print(loss)

11

tensor(0.5959, grad_fn=<AddBackward0>)